In [1]:
# 과제3: 결정 트리와 랜덤 포레스트를 사용한 moons 데이터셋 분류

import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 1. 데이터 준비
# make_moons 함수로 10,000개의 샘플을 가진 데이터셋 생성 (노이즈 0.4 추가)
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

# 데이터를 훈련 세트(80%)와 테스트 세트(20%)로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. 결정 트리 하이퍼파라미터 최적화
# 결정 트리 분류기 초기화
clf = DecisionTreeClassifier(random_state=42)

# GridSearchCV로 최적의 하이퍼파라미터를 찾기 위한 파라미터 설정
param_grid = {
    'max_depth': [5, 6, 10, 15, 20],
    'max_leaf_nodes': [5, 10, 17, 20, 50],
    'min_samples_split': [2, 5, 10]
}

# GridSearchCV를 사용하여 최적의 파라미터 탐색
# 5-fold 교차 검증을 사용하여 각 파라미터 조합 평가
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", grid_search.best_params_)

# 최적의 하이퍼파라미터로 훈련된 모델 평가
best_clf = grid_search.best_estimator_  # 최적의 모델 가져오기
y_pred = best_clf.predict(X_test)      # 테스트 데이터에 대한 예측 수행
single_tree_accuracy = accuracy_score(y_test, y_pred)  # 정확도 계산
print("Single Tree Accuracy:", f"{single_tree_accuracy:.4f}")

# 3. 랜덤 포레스트 구현
# 서브샘플의 개수 설정
n_trees = 100  # 랜덤 포레스트를 구성할 결정 트리 개수
n_instances = X_train.shape[0]  # 훈련 데이터의 샘플 수

# 각 트리에서 사용할 서브샘플 생성 및 모델 훈련
forest = []  # 랜덤 포레스트를 구성할 트리 저장 리스트
accuracies = []  # 각 트리의 정확도를 저장할 리스트

for _ in range(n_trees):
    # 훈련 데이터에서 샘플링 (중복 허용, 즉 부트스트랩 샘플링)
    indices = np.random.randint(0, n_instances, n_instances)  # 랜덤 인덱스 생성
    X_subset, y_subset = X_train[indices], y_train[indices]   # 서브샘플 데이터 추출

    # 결정 트리 초기화 및 훈련
    tree_clf = DecisionTreeClassifier(max_depth=None, max_leaf_nodes=20, min_samples_split=2, random_state=None)
    tree_clf.fit(X_subset, y_subset)

    # 테스트 데이터로 예측 수행 및 정확도 저장
    y_pred = tree_clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)  # 정확도 리스트에 추가

    forest.append(tree_clf)  # 트리를 포레스트 리스트에 추가

# 평균 단일 트리 정확도 출력
average_tree_accuracy = np.mean(accuracies)  # 각 트리 정확도의 평균 계산
print("Average Single Tree Accuracy:", f"{average_tree_accuracy:.4f}")

# 4. 다수결 앙상블
# 테스트 데이터에 대한 모든 트리의 예측값 수집
predictions = np.zeros((n_trees, X_test.shape[0]), dtype=np.int64)  # 모든 트리의 예측값을 저장할 배열

for i, tree in enumerate(forest):
    predictions[i] = tree.predict(X_test)  # 각 트리의 예측값 저장

# 다수결 방식으로 최종 예측 생성
# axis=0을 기준으로 각 샘플에 대해 가장 많이 등장한 클래스 선택
final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)

# 다수결 앙상블 모델의 정확도 계산
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print("Ensemble Accuracy:", f"{ensemble_accuracy:.4f}")

Best Parameters: {'max_depth': 10, 'max_leaf_nodes': 20, 'min_samples_split': 2}
Single Tree Accuracy: 0.8700
Average Single Tree Accuracy: 0.8643
Ensemble Accuracy: 0.8700


Single Tree Accuracy: 최적 하이퍼파라미터를 사용한 단일 결정 트리가 테스트 셋에서 87%의 정확도가 나타남
결정 트리가 데이터의 주요 패턴을 잘 학습했음을 나타내지만 단일 트리는 랜덤 샘플링에 민감하고, 데이터의 특정 부분에 과적합될 가능성

Average Single Tree Accuracy: 랜덤 샘플링된 100개의 서브셋에 대해 훈련된 개별 결정 트리의 정확도를 평균 낸 값
단일 트리 정확도(87%)보다 약간 낮은 이유는 각 트리가 훈련 데이터의 서브셋만 사용하기 때문에, 데이터의 전체적인 분포를 완전히 반영하지 못하기 때문

Ensemble Accuracy: 다수결 앙상블(Random Forest) 방식을 사용한 최종 모델의 정확도
랜덤 포레스트의 다수결 앙상블은 개별 트리의 예측 편향을 줄이고, 더 높은 일반화 성능을 제공

단일 결정 트리와 앙상블 모델의 성능 차이는 나타나지 않았는데,
이는 make_moons 데이터셋이 비교적 간단한 문제이고, 최적화된 단일 트리도 이미 상당히 높은 성능을 제공하기 때문에 발생

적절한 max_depth와 max_leaf_nodes 설정은 모델의 복잡도를 제어하여, 과적합을 방지하면서도 데이터를 충분히 학습하도록 도우며,
min_samples_split의 기본값(2)은 이 데이터셋의 특성상 추가적인 최적화 없이도 좋은 성능을 제공한다.